并发（concurrency）一直是计算机里常见的现象。理解并发的关键在于不同的任务在执行的时候，相互之间存在重叠(overlap)。一个任务的**开始时间（start）**和**结束时间（end）** 构成了一个区间（interval）。如何处理这种区间和重叠成为了Leetcode上一类有意思的问题。合并（merge）区间，或是找出不相交的区间等等。既有对逻辑思维的训练，也面向了实际生活面临的问题。下面就这类问题进行简单的梳理。

### 区间关系

给定两个区间（interval），例如 A 和 B，它们的关系无怪乎就下面图示的六种情况

![区间关系图](https://s2.ax1x.com/2020/01/07/lcYwNT.png)

需要特别注意前三种情况，往往很多问题最终都可以简化成处理这三种情况的case。 即 A.start <= B.start 。有两个重要的性质

对于 2， 3 情况下的 overlap 区域，可以表示为

```
start = max(A.start, B.start)  
end = min(A.end, B.end)


```

`[start, end]` 就是重叠的区域。如果 `end <= start`，那么就是图示中第一种情况，A 和 B 并不重叠。

> 注意，对于临界条件，根据具体的问题判断

重叠的部分是为**交集**，在集合的概念里，还有另外一个运算叫**并集**。并集的运算代码如下

```
start = min(A.start, B.start)
end = max(A.start, B.end)
```

由于前提是 A.start <= B.start，处理 start 的时候就比较简单。同时也暗示的一个信息，就是在处理 intervals 的时候，如果没有什么思路，可以把 interval 按照 其 start 进行排序。排序之后颇有柳暗花明的效果。当然，时间复杂度最好也是 O(NlogN)。

### 区间合并（merge intervals）

上面的提到的 interval 求并集，Leetcode上第[56. 合并区间](https://leetcode-cn.com/problems/merge-intervals)题


> 给出一个区间的集合，请合并所有重叠的区间。
> 
> 示例 1:
>
> 输入: [[1,3],[2,6],[8,10],[15,18]]
> 
> 输出: [[1,6],[8,10],[15,18]]
> 
> 解释: 区间 [1,3] 和 [2,6] 重叠, 将它们合并为 [1,6].
> 
> 示例 2:
>
> 输入: [[1,4],[4,5]]
>
> 输出: [[1,5]]
>
> 解释: 区间 [1,4] 和 [4,5] 可被视为重叠区间。

从题意可知，给定的输入是一个二维 list。list 的每个元素是一个 interval，需要的就是求这些 interval的merge并集。处理的过程类似 reduce，过程。假设 第一项是A， 第二项是B，A和B处理之后成为新的 A，那么接下来的一项是新的 B，依次类推，直到处理完list所有元素。

既然是处理 A 和 B 的问题，那么就可以套用上面所说的两个方法，当然前提是需要针对 list 中的 interval的 start 进行排序。

如下图所示

![图](https://s2.ax1x.com/2020/01/07/lc26YQ.md.png)


经过排序之后，A B C 的三者中的两者可以归结为前面所述的 1， 2， 3 中类型。

A[1, 4] 和 C[7, 9] 是第一种情况, 两者不相交

A[1, 4] 和 B[2, 5] 是第二种情况，有重叠的部分

对于第二种类型，直接进行 merge 求并集即可。求完并集之后需要创建一个新的 A'[1, 5]，但是这个 A' 暂时不能放到结果中。因为 这个 A' 如果 end 很大，它跟 C 也有可能组成 第二种情况，即上图的中的

A [1, 8] merge B[2, 5] 得到的 A'[1, 8]，其中 A' 和 C 又有重叠，需要merge。

只有两个 interval的 组合是第一种情况，即无交集的时候，才把当前的 start 和 end 更新到结果中。 具体代码如下:

In [3]:
from typing import *

class Solution:
    def merge(self, intervals: List[List[int]]) -> List[List[int]]:
        if len(intervals) < 2:
            return intervals
        
        intervals.sort(key=lambda x: x[0])
        
        ret = []
        start, end = intervals[0][0], intervals[0][1]
        for istart, iend in intervals[1:]:
            if istart <= end:
                end = max(end, iend)
            else:
                ret.append([start, end])
                start, end = istart, iend
        
        ret.append([start, end])
        return ret
        
intervals = [[1, 3],[2, 6],[8, 10],[15, 18]]
ret = Solution().merge(intervals)
assert ret == [[1, 6], [8, 10], [15, 18]], f'{ret} is err'

解法就是针对排序后的list进行迭代区间merge处理。使用一对 start，end 标记当前 merge 后的 A'，使用贪心的思想，假设 A' 能和接下来的 interval 进行merge。处理下一个 interval 的时候，两者的关系无非是重叠和非重叠，判断重叠的方式使用求交集的方式。并更新 end。如果没有重叠，则记录目标的解，同时也要更新 start end，即 A'。

需要注意的是，迭代结束之后，对于当前的 A', 其实也是一个解，不能忘记追加到结果集中。

因为需要先对输入的 list 进行排序，排序的时间复杂度为 O(NlogN)。排序之后，对list 进行迭代 merge 处理，一共所需要的时间也就是迭代的时间。因此最终的时间复杂度是 O(NlogN)。

空间方面主要是结果集的存储，即没有任何交集的情况下，需要返回的就是输入值。因此需要 O(N)的空间。当然，对于某些 排序方法，也需要额外的空间。取决于具体的语言和平台。

### [435. 无重叠区间](https://leetcode-cn.com/problems/non-overlapping-intervals)

区间合并求重叠是一类问题，其对应的有一种处理方式就是去除重叠部分。如 Leetcode的 435 题：

> 给定一个区间的集合，找到需要移除区间的最小数量，使剩余区间互不重叠。
>
> 注意:
>
> 可以认为区间的终点总是大于它的起点。区间 [1,2] 和 [2,3] 的边界相互“接触”，但没有相互重叠。
>
> 示例 1:
> 
> 输入: [ [1,2], [2,3], [3,4], [1,3] ]
> 
> 输出: 1
> 
> 解释: 移除 [1,3] 后，剩下的区间没有重叠。
>
> 示例 2:
> 
> 输入: [ [1,2], [1,2], [1,2] ]
> 
> 输出: 2
> 
> 解释: 你需要移除两个 [1,2] 来使剩下的区间没有重叠。
>
> 示例 3:
> 
> 输入: [ [1,2], [2,3] ]
> 
> 输出: 0
> 
> 解释: 你不需要移除任何区间，因为它们已经是无重叠的了。


与前面介绍的套路一样，首先对 list 进行排序。然后使用贪心的策略，即取列表的第一项为当前的A'。让其跟下一个区间进行比对，如果有重叠，更新除去的计数器。如果没有重叠，就移动当前的 A'。代码和上面一题非常相似，如下：


In [8]:
from typing import *

class Solution:
    def eraseOverlapIntervals(self, intervals: List[List[int]]) -> int:
        if not intervals:
            return 0
        
        intervals.sort(key=lambda x: x[0])
        
        end = intervals[0][1]
        ret = 0
        for istart, iend in intervals[1:]:
            if istart < end:
                ret += 1
                end = min(end, iend)
            else:
                end = iend
        return ret

intervals = [ [1,2], [2,3], [3,4], [1,3] ]  # 1 [1, 3]
intervals = [ [1,2], [1,2], [1,2] ]         # 2, [1, 2], [1, 2]
intervals = [ [1,2], [2,3] ]                # 0
intervals = []
ret = Solution().eraseOverlapIntervals(intervals)
print(ret)

0


需要注意的是，56题合并区间使用的求并集的算法，所以更新的 end = max(end, iend) ，而这道题其实要求的是区间的交集（保留重叠少的部分），所以更新的 end = min(end, iend)。其中原委不难，但需细品。

因为也是需要先排序，然后再迭代intervals，最终的算法时间复杂度是 O(NlogN)，没有使用额外的空间，空间复杂度为 O(1)。


### 插入区间

了解了区间重叠的交集和并集技巧，可以处理不少类似模式（pattern）的题目。即使最开始的问题并不是求解两个 intervals的merge操作，也可以通过迭代逐步的转化。

[57. 插入区间](https://leetcode-cn.com/problems/insert-interval)

> 给出一个无重叠的 ，按照区间起始端点排序的区间列表。
> 
> 在列表中插入一个新的区间，你需要确保列表中的区间仍然有序且不重叠（如果有必要的话，可以合并区间）。
> 
> 示例 1:
> 
> 输入: intervals = [[1,3],[6,9]], newInterval = [2,5]
> 
> 输出: [[1,5],[6,9]]
> 
> 示例 2:
> 
> 输入: intervals = [[1,2],[3,5],[6,7],[8,10],[12,16]], newInterval = [4,8]
> 
> 输出: [[1,2],[3,10],[12,16]]
> 
> 解释: 这是因为新的区间 [4,8] 与 [3,5],[6,7],[8,10] 重叠。


与56题合并区间类似，上面的题目要求插入一个 interval，同时插入之后整个 intervals 依然保持互相不重叠。前提是输入本身就是一个有序且无重叠的intervals。

如果被插入的 interval 跟现有的intervals列表没有重叠.那么最简单不过，直接找到排序位置插入即可。如下图，在 `[1, 3] [6, 8] [9, 10]` intervals 中插入 interval `[4, 5]`，找到空隙插入即可，空隙就是被插入的interval的start 要大于上一个 interval 的end。

![图片](https://s2.ax1x.com/2020/01/08/lgwy6S.md.png)

如果跟现有的重叠呢？很明显，需要跟现有的合并。使用贪心的策略，假设被插入的interval 很大，那么就有可能 merge 这个输入的intervals。即下图的两种情况

![图片](https://s2.ax1x.com/2020/01/08/lgwL79.md.png)

对于被插入的 interval 是 `[4, 7]`，那么它与 `[6, 8]` 重叠，因此需要 merge 合并成 `[4, 8]` 再插入。因为当前的 end = 8，小于下一个 interval `[9,10]`的start，因此可以直接插入并终止算法。

可是对于 `[4, 10]` 这个区间，即使跟 `[6, 8]`merge之后，它依然和下一个  interval `[9,10]`存在重叠，因此需要继续迭代合并`[9,10]`，直到出现可以插入的位置。

所以求解此题的一个思路就是通过迭代排序的 intervals，找到重叠的 interval，然后进行合并，合并之后再判断是否满足题意，以最终求解返回。

剩下的就是代码实现了：



In [9]:
from typing import *


class Solution:
    def insert(self, intervals: List[List[int]], newInterval: List[int]) -> List[List[int]]:
        start, end = newInterval[0], newInterval[1]
        ret = []
        i = 0
        # 寻找可以插入的起始位置，与上一个 interval 不重叠
        while i < len(intervals):
            istart, iend = intervals[i][0], intervals[i][1]
            if iend < start:
                ret.append([istart, iend])
            else:
                break
            i += 1
        
        # 处理当前重叠的 interval，merge 所有
        while i < len(intervals):
            istart, iend = intervals[i][0], intervals[i][1]
            if end < istart:
                break
            
            # 使用了求 并集 的技巧
            start = min(start, istart)   
            end = max(end, iend)
            i += 1

        # 插入不能合并区间
        ret.append([start, end])
        
        # 将剩余的结果也保持到结果集中
        while i < len(intervals):
            ret.append(intervals[i])
            i += 1
        
        return ret
            

intervals = [[1,3],[6,9]]
newInterval = [2,5]

intervals = [[1,3], [5,7], [8,12]]
newInterval = [4, 6]

intervals = [[1,3], [5,7], [8,12]]
newInterval = [4, 10]

intervals = [[2,3], [5, 7]]
newInterval = [1, 4]

intervals = [[1,5]]
newInterval = [2, 3]

intervals = []
newInterval = [2, 3]

ret = Solution().insert(intervals, newInterval)
print(ret)

[[2, 3]]


上述的代码一共分为三步

1. 找到上一个 interval 和 被插入的 interval 不重叠的位置，即`pre_interval.end <= interval.start`，插入之前不重叠的 interval。
2. 找到待 merge 的interval，依次merge，使用的技巧类似 56 题的 区间合并技术。如果不再重叠，就将结果保存 `ret.append([start, end])`
3. 合并区间之后，将剩余输入的 interval 保存到结果集中。

输入的intervals已经是有序的，处理的过程是直接迭代。最终的时间复杂度是 O(N)，空间复杂度也是 O(N)，需要将结果保存输出。


### 986. 区间列表的交集

面对一堆 intervals，最终也是转换成最基本的 A B 两个interval进行处理，而处理两者关系的时候，往往又使用到贪心策略。无论多么复杂，都可以按照这个思路先思考。下面一题输入是两个区间列表，其实最终也是一样的策略。

[986. 区间列表的交集](https://leetcode-cn.com/problems/interval-list-intersections/)

> 给定两个由一些闭区间组成的列表，每个区间列表都是成对不相交的，并且已经排序。返回这两个区间列表的交集。（形式上，闭区间 [a, b]（其中 a <= b）表示实数 x 的集合，而 a <= x <= b。两个闭区间的交集是一组实数，要么为空集，要么为闭区间。例如，[1, 3] 和 [2, 4] 的交集为 [2, 3]。）
>
> 示例：
>
> ![图](https://assets.leetcode-cn.com/aliyun-lc-upload/uploads/2019/02/02/interval1.png)
>
> 输入：A = [[0,2],[5,10],[13,23],[24,25]], B = [[1,5],[8,12],[15,24],[25,26]]
> 
> 输出：[[1,2],[5,5],[8,10],[15,23],[24,24],[25,25]]
> 
> 注意：输入和所需的输出都是区间对象组成的列表，而不是数组或列表。


题意给的是两个 intervals列表并且都是已经排序的，一种思路是先合并两个list，将其打平为一个列表，然后再排序，就转变成了类似 56 题的模式。或者直接使用暴力方式，一次迭代处理两个列表元素的重叠merge情况。代码如下：

In [10]:
from typing import *


class Solution:
    def intervalIntersection(self, A: List[List[int]], B: List[List[int]]) -> List[List[int]]:
        ret = []
        for i in range(len(A)):
            for j in range(len(B)):
                if B[j][1] < A[i][0]: 
                    continue
                if A[i][1] < B[j][0]:
                    continue
                ret.append([max(A[i][0], B[j][0]), min(A[i][1], B[j][1])])     
        return ret

通常在 leetcode上，暴力破解都不是有效的解法。面对两个列表，可以使用 双指针，即每个列表一个指针，然后处理对应的 interval的重叠交集，然后再移动指针。这些循环结果是线性复杂度。

代码如下：

In [11]:
# 双指针 O(N+M)  overlap的重要性质  start = max(A.start, B.start) end = min(A.end, B.end) start <= end
        

from typing import *


class Solution:
    def intervalIntersection(self, A: List[List[int]], B: List[List[int]]) -> List[List[int]]:
        ret = []
        i = j = 0
        while i < len(A) and j < len(B):
            start = max(A[i][0], B[j][0])
            end = min(A[i][1], B[j][1])
            
            # 对于重叠的interval，求交集
            if start <= end:
                ret.append([start, end])     
            
            # 移动指针，无论重叠还是不重叠，移动指针都是将 end 小的移动
            if A[i][1] < B[j][1]:
                i += 1
            else:
                j += 1     
        return ret
                    
A = [[1, 3], [5, 6], [7, 9]]
B = [[2, 3], [5, 7]]

A = [[1, 3], [5, 7], [9, 12]]
B = [[1, 2]]

A = [[0,2],[5,10],[13,23],[24,25]]
B = [[1,5],[8,12],[15,24],[25,26]]

A = [[10,50],[60,120],[140,210]]
B = [[0,15],[60,70]]
ret = Solution().intervalIntersection(A, B)
print(ret)

[[10, 15], [60, 70]]


从代码可以看出重叠的重要特性是通过求交集的技巧，找到 A B 的第一种关系的条件。一旦处理了当前指针的区间，就需要移动指针，移动的技巧就是找 end 小的部分。当前 end 小，那么下一个 interval的 start 就有可能小，end的大的就有可能重叠下一个 start 小的，还是贪心的策略。

时间复杂度是 O(N+M)，两个列表长度分别是 N 和 M。

与之类似的还有下面一题：

### 1229. 安排会议日程

前面有所提及，区间问题最常见来着并发程序的调度。现实生活中也经常有这样的情形，时间空暇或者工作时间都是一个个区间。休息和工作的关系无非就是区间的重叠问题。1229就是一道贴近日常工作的问题。

[1229. 安排会议日程](https://leetcode-cn.com/problems/meeting-scheduler/)

> 你是一名行政助理，手里有两位客户的空闲时间表：slots1 和 slots2，以及会议的预计持续时间 duration，请你为他们安排合适的会议时间。
> 
> 「会议时间」是两位客户都有空参加，并且持续时间能够满足预计时间 duration 的 最早的时间间隔。
> 
> 如果没有满足要求的会议时间，就请返回一个 空数组。
> 
> 「空闲时间」的格式是 [start, end]，由开始时间 start 和结束时间 end 组成，表示从 start 开始，到 end 结束。
> 
> 题目保证数据有效：同一个人的空闲时间不会出现交叠的情况，也就是说，对于同一个人的两个空闲时间 [start1, end1] 和 [start2, end2]，要么 start1 > end2，要么 start2 > end1。
> 
> 示例 1：
> 
> 输入：slots1 = [[10,50],[60,120],[140,210]], slots2 = [[0,15],[60,70]], duration = 8
> 
> 输出：[60,68]
> 
> 示例 2：
> 
> 输入：slots1 = [[10,50],[60,120],[140,210]], slots2 = [[0,15],[60,70]], duration = 12
> 
> 输出：[]


与上一题的求解策略一样，也是先找出两个区间列表的交集，然后再判断这个交集长度是否满足第二个参数 duration。


In [12]:
class Solution:
    def minAvailableDuration(self, A: List[List[int]], B: List[List[int]], duration: int) -> List[int]:
        A.sort(key=lambda x: x[0])
        B.sort(key=lambda x: x[0])
        
        i = j = 0
        while i < len(A) and j < len(B):
            start = max(A[i][0], B[j][0])
            end = min(A[i][1], B[j][1])
            
            if end - start >= duration:
                return [start, start + duration]
            
            if A[i][1] <= B[j][1]:
                i += 1
            else:
                j += 1
                
        return []
    

A = [[10,50],[60,120],[140,210]]
B = [[0,15],[60,70]]
duration = 8

A = [[10,50],[60,120],[140,210]]
B = [[0,15],[60,70]] 
duration = 12

ret = Solution().minAvailableDuration(A, B, duration)
print(ret)

[]


时间复杂度也是 O(N+M)。解题的关键还是转换成两个区间 A 和 B 的进行求交集。

看了HR 安排日常也需要一定的算法知识，不然可能就是人工的枚举暴力破解。Leetcode或者 geeksforgeeks 上还有几道贴近生活的区间问题。

### 会议室

[252.会议室](https://leetcode-cn.com/problems/meeting-rooms)
 
> 给定一个会议时间安排的数组，每个会议时间都会包括开始和结束的时间 [[s1,e1],[s2,e2],...] (si < ei)，请你判断一个人是否能够参加这里面的全部会议。
> 
> 示例 1:
> 
> 输入: 
> [[0,30],[5,10],[15,20]]
> 
> 输出: false
> 
> 示例 2:
> 
> 输入: [[7,10],[2,4]]
> 
> 输出: true

经过了上面几个题目的训练，想必这个问题就迎刃而解了。会议时间是区间，场景就是日常的工作情况。能参加所有会议，那么就是区间都没有重叠的即可。

代码如下

In [13]:
from typing import *

class Solution(object):
    def canAttendMeetings(self, intervals: List[List[int]]) -> bool:
        intervals.sort(key=lambda x: x[0])
        end = intervals[0][1]
        for istart, iend in intervals[1:]:
            if istart < end:
                return False
            else:
                end = iend
        return True
        
    
intervals = [[0,30], [5,10], [15,20]]
intervals =  [[7,10],[2,4]] 
ret = Solution().canAttendMeetings(intervals)
print(ret)


True


这题比较简单，它的升级版 [253.会议室II](https://leetcode-cn.com/problems/meeting-rooms-ii/) 则需要一定的技巧。这是一种新的模式（pattern）

> 题目描述 
> 给定一个会议时间安排的数组，每个会议时间都会包括开始和结束的时间  [[ s1 , e1 ] ，[ s2 , e2 ]，…] (si < ei) ，为避免会议冲突，同时要考虑充分利用会议室资源，请你计算至少需要多少间会议室，才能满足这些会议安排。
> 
> 示例 1:
> 
> 输入: [[0, 30],[5, 10],[15, 20]]
> 
> 输出: 2
> 
> 示例 2:
> 
> 输入: [[7,10],[2,4]]
> 
> 输出: 1


日常生活也是这样，往往不同的会议再相同的时间举行，就得需要 HR 安排不同的会议室了。转换成 区间算法问题，无非就是判断这些区间是否有重叠。初看还是老问题套路，可是实际想想，两个列表的交集的前提是我们知道有两个列表的输入，此题虽然也知道一共列表的大小。但是我们总不能初始化这些长度的指针吧。

仔细想想，假设就是一个HR，不管任何算法。常人的思维也是一种贪心策略：

1. 第一个会肯定需要占一个会议室，因此初始化一个会议室放第一个区间。
2. 然后看第二个会，如果跟第一个会议有冲突，那么就再安排一个会议室
3. 再看第三个会议，前面已经有两个会议室了，如果有空的，即轮到它的时候，别的会议都结束了，会议室为空，就安排，没有就再初始化一个会议室。

关键是2 3 步骤的时候。如果当前是有空的会议室，安排到那个会议室呢？在代码里，我们可以扫描一遍当前的会议室，有合适的就安排。那么时间复杂度会浪费再扫描会议室上。

一个行之有效的策略就是，当前最早结束的会议如果空了，那么就可以安排，如果没有，就看下一个最早结束的，直到扫描所有会议室。看起来还是需要扫描所有会议室，但是比之前的线性扫描，可以提前结束扫描。

既然扫描的过程中需要最早结束的会议室，那么这个会议室就可以组织成 堆(heap) 这种数据结构。找最大最小只需要常数时间，而且堆的增删过程中，维护堆的性质也不过是O(logN) 。

代码如下：

In [14]:
# 最小堆 O(NlogN)

from typing import *
from heapq import heappop, heappush

class Solution:
    def minMeetingRooms(self, intervals: List[List[int]]) -> int:
        if not intervals:
            return 0
        
        rooms = []
        intervals.sort(key=lambda x: x[0])
        
        for i in intervals:
            if len(rooms) > 0 and rooms[0] <= i[0]:
                heappop(rooms)
            heappush(rooms, i[1])
        return len(rooms)

intervals = [[1,4], [2,5], [7,9]]
intervals = [[6,7], [2,4], [8,12]]
intervals = [[1,4], [2,3], [3,6]]
intervals = [[4,5], [2, 3], [2, 4], [3, 5]]

ret = Solution().minMeetingRooms(intervals)
print(ret)

2


依然是老套路

1. 先以 interval.start 排序 intervals
2. 然后迭代 interval，堆为空，则把当前的 interval 加入堆。表示安排一个会议室给这个 interval。如果堆不为空，且最小堆的end比interval.start 还小，说明当前的 interval可以在这个最小堆后面的会议室。即可以替换这个最小堆的元素。代码只需要弹出这个最小堆堆顶元素。
3. 再把当前的 interval 加入堆中。

最终堆的大小，就是所需要的最小会议室。

排序的时间复杂度是 O(NlogN), 迭代处理堆的操作是 O(logN)，最终的复杂度还是 O(NlogN)。空间复杂度则是建堆的空间，即 O(N)。


### 员工空闲时间

引入最小堆，是因为当前需要处理的 interval，需要跟上一个最小的 interval进行比较。下面的一个问题，也是需要接祖 最小堆 进行处理。

> 我们得到了一个雇员的时间表，它代表了每个雇员的工作时间。每个员工工作时间是一个不重叠区间的列表，并且这些区间是按照排序的顺序排列的。
> 返回有限时间列表，表示所有员工的公共、正长度空闲时间且排序的有限时间列表
> 
> 示例一:
> 
> 输入: schedule = [[[1,2],[5,6]],[[1,3]],[[4,10]]]
> 输出: [[3,4]]> 
> 释: 总共有三个雇员，所有公共的空闲时间间隔都是[-inf，1] ，[3,4] ，[10，inf ]。 我们抛弃任何包含 inf 的区间，因为它们不是有限的。
> 
> 示例二:
> 
> 输入: schedule = [[[1,3],[6,7]],[[2,4]],[[2,5],[9,12]]]
> 输出: [[5,6],[7,9]]
> 
> (即使我们以[ x，y ]的形式表示时间区间，但其中的对象是时间区间，而不是列表或数组。 例如，schedule [0][0]。 开始1，进度表[0][0]。 结束2，计划[0][0][0]没有定义。)
> 
> 此外，我们不会在回答中包含像[5,5]这样的区间，因为它们的长度为零。


从题意可知，求员工的空闲时间，也就是求员工工作时间的非集之间的交集。可以先求出所有员工的interval的非集，然后再求其交集，问题就转换为 986. 区间列表的交集。

当然，熟悉集合运算的话，这个问题也可以转换成所有员工的 interval的交集，然后再求其非集。如下图所示

![图](https://s2.ax1x.com/2020/01/08/l2Pc7t.md.png)


员工一的 时间为 `[1, 3], [9, 12]`

员工二的 时间为 `[2, 4]`

员工三的 时间为 `[6, 8]`

三个员工的工作时间interval的merge 就是 

`[1, 4] [6, 8], [9, 12]` 这个新的intervals的间隙（gap），恰好就是所要求的结果，也就是merge之后的interval的非集合，即 `[4, 6] 和 [8, 9]`。、

代码如下


In [15]:
from typing import *
from heapq import heappop, heappush

class Solution:
    
    def employeeFreeTime(self, schedule: List[List[int]]) -> List[List[int]]:
        if len(schedule) == 0:
            return []
        
        ret = []
        s = []
        for i in schedule:
            for j in i:
                heappush(s, j)
                
        start, end = s[0][0], s[0][1]
        while len(s) > 0:
            interval = heappop(s)
            istart, iend = interval[0], interval[1]
            if istart <= end:
                end = max(end, iend)
            else:
                ret.append([end, istart])
                start, end = istart, iend
        return ret

schedule = [[[1,2],[5,6]],[[1,3]],[[4,10]]]         # [[3,4]]
schedule = [[[1,3],[6,7]],[[2,4]],[[2,5],[9,12]]]   # [[5,6],[7,9]]
schedule = [[[1,3], [5,6]], [[2,3], [6,8]]]         # [3, 5]
schedule = [[[1,3], [9,12]], [[2,4]], [[6,8]]]      # [[4, 6], [8, 9]]
schedule = [[[1,3]], [[2,4]], [[3,5], [7,9]]]       # [[5, 7]]
ret = Solution().employeeFreeTime(schedule)
print(ret)

[[5, 7]]


考虑到需要将所有员工的 interval 进行排序，输入又是嵌套列表。因此可以借助最小堆。迭代 所有员工的时间，建一个最小堆。

这个堆里的 元素可能存在有重叠的，依次在循环中弹出堆元素，如果当前记录的 start， end 和上一个 interval存在重叠，就进行合并，并更新 start end，这个处理和 56题一样。

如果不重叠，则表示他们之间有 gap，记录这个 gap到结果集中，同时更新 start，end。当堆元素都pop完毕，当前的 start 和 end 是所以merge合并的并集，因此不需要加入结果集中。

上述的算法中，堆排序需要的时间复杂度是 O(NlogM)，M是员工的个数，迭代堆的操作是线性，堆pop的操作调整是 O(logM)，最终的复杂度也是 O(NlogM)，

至此，大部分关于 interval merge 的技巧都介绍了。剩下的就是如何综合的使用这些技巧，例如 geeksforgeeks 有一道题目


最大CPU负载，cpu负载对于程序员来说是再熟悉不过了。计算某个时间的最大负载算法就用到了区间相关技术，题目为

> 我们得到了一份乔布斯的名单。 每个作业在运行时都有一个开始时间、一个结束时间和一个 CPU 负载。 我们的目标是，如果所有作业都在同一台机器上运行，那么在任何时候都能找到最大的 CPU 负载。
>
> 示例一:
> 输入: [[1,4,3], [2,5,4], [7,9,6]]
> 输出: 7
> 说明: 区间 [1,4,3] 和 [2,5,4] 有重叠, 那么最大 CPU load (3+4=7) ，另外一个区间 [7, 9] 的load仅仅是 6
> 
> 示例二:
> 输入: [[6,7,10], [2,4,11], [8,12,15]]
> 输出: 15
> 说明: 没有重叠的区间，最大的 load 就是最后一个区间的load，15.
> 
> 示例三:
> 输入: [[1,4,2], [2,4,1], [3,6,5]]
> 输出: 8
> 说明: 三个区间都有重叠，最大的load就是三个区间的load之和 8

这一题贴近工作，但是初看也会有点迷糊。仔细思考一下，无非还是需要求所有 interval的merge交集，在求merge交集的同时计算 merge 的第三个参数，即 cpu的load。

那么就转换成了[253.会议室II](https://leetcode-cn.com/problems/meeting-rooms-ii/)。

使用一个最小堆存储有重叠的 interval，并且计算 重叠的interval的load值为当前的load值 cur_load。当迭代下一个interval 依次和堆中元素进行比较，如果没有重叠，就说明跟这些区间没有关系。然后依次出堆，同时减掉之前用来计数的 cur_load。然后再把当前的interval 入堆，并且计算更新 cur_load 和结果 max_load。

具体代码如下：


In [16]:
### Maximum CPU Load 

from heapq import heappop, heappush

class Solution:
    
    def maximumCPULoad(self, jobs: List[List[int]]) -> int:
        jobs.sort(key=lambda x: x[0])
        
        max_load = 0
        cur_load = 0
        min_heap = []
        for i in jobs:
            while len(min_heap) > 0 and min_heap[0][1] <= i[0]:
                job = heappop(min_heap)
                cur_load -= job[2]
            heappush(min_heap, i)
            cur_load += i[2]
            max_load = max(max_load, cur_load)
        return max_load
        
        
jobs = [[1,4,3], [2,5,4], [7,9,6]]
jobs = [[6,7,10], [2,4,11], [8,12,15]]
jobs = [[1,4,2], [2,4,1], [3,6,5]]

ret = Solution().maximumCPULoad(jobs)
print(ret)

8


### 总结

区间合并（interval merge）相关的问题大抵如此。首先需要熟悉两个 interval 之间的关系。开篇介绍了 6 种关系。归纳一下，实际上就三种，1 不重叠；2. 重叠不包含；3. 重叠包含。另外三种是这三种的镜像。对于不重叠的区间，有时候需要学会如何求他们的 gap。对于重叠的区间，有两个算法，就交集和就并集。通常在解决一个 interval是列表的时候，可以优先按照 interval.start 进行排序。然后迭代的过程中，把问题简化成处理 A 和 B 两个interval的关系，无非是 merge 求交集和并集。在输出最后结果的时候，注意贪心策略，求并集的时候，当前的并集未必是整个 列表的并集，需要依次迭代，直到有 gap的区间。另外，对于合并多个列表的区间，也是通过双指针的方式转换处理 A 和 B 的问题，然后再判断移动哪个指针。

最后就是处理当前的 interval可能与上一个，甚至是前几个 interval 进行比较，通常可以借助 堆 数据结构处理比较，既能保证顺序，又有时间复杂度上的优势。

总而言之，化繁为简，多加练习。

PS：leetcode 上的这类题，有不少都需要会员。。。

> [56. 合并区间](https://leetcode-cn.com/problems/merge-intervals)
> 
> [435. 无重叠区间](https://leetcode-cn.com/problems/non-overlapping-intervals)
> 
> [57. 插入区间](https://leetcode-cn.com/problems/insert-interval)
> 
> [986. 区间列表的交集](https://leetcode-cn.com/problems/interval-list-intersections/)
> 
> [1229. 安排会议日程](https://leetcode-cn.com/problems/meeting-scheduler/)
> 
> [252.会议室](https://leetcode-cn.com/problems/meeting-rooms)
>  
> [253.会议室II](https://leetcode-cn.com/problems/meeting-rooms-ii/) 
> 
> [759. 员工空闲时间](https://leetcode-cn.com/problems/employee-free-time)
>
> 来源：力扣（LeetCode） [https://leetcode-cn.com/problemset/all/](https://leetcode-cn.com/problemset/all/)